In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import random
import re

In [2]:
data = pd.read_csv('./final_data/final_data.csv')
print(data.shape)
data.columns

(84432, 44)


Index(['race', 'gender', 'age', 'admission_type_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'acetohexamide',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'citoglipton', 'insulin',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'f_diag', 's_diag', 't_diag',
       'clustering3', 'clustering4', 'clustering2', 'medical_specialty_new',
       'discharge_id', 'admission_id'],
      dtype='object')

In [3]:
data = data.sort_values('target', ascending=True)

In [10]:
cat_cols = list(data.select_dtypes(include=[object]).columns)
for col in cat_cols:
    data[col] = pd.Categorical(data[col])

In [11]:
data.dtypes

race                      category
gender                    category
age                       category
admission_type_id         category
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum             category
A1Cresult                 category
metformin                 category
repaglinide               category
nateglinide               category
chlorpropamide            category
acetohexamide             category
glipizide                 category
glyburide                 category
tolbutamide               category
pioglitazone              category
rosiglitazone             category
acarbose                  category
miglitol                  category
citoglipton               category
insulin                   category
metformin-pioglitazo

In [13]:
le = preprocessing.LabelEncoder()
col_to_encode = data[list(data.select_dtypes(include=['category']).columns)]
for col in col_to_encode:
    data[col] = le.fit_transform(data[col])

In [14]:
data.dtypes

race                      int32
gender                    int32
age                       int32
admission_type_id         int64
time_in_hospital          int64
num_lab_procedures        int64
num_procedures            int64
num_medications           int64
number_outpatient         int64
number_emergency          int64
number_inpatient          int64
number_diagnoses          int64
max_glu_serum             int32
A1Cresult                 int32
metformin                 int32
repaglinide               int32
nateglinide               int32
chlorpropamide            int32
acetohexamide             int32
glipizide                 int32
glyburide                 int32
tolbutamide               int32
pioglitazone              int32
rosiglitazone             int32
acarbose                  int32
miglitol                  int32
citoglipton               int32
insulin                   int32
metformin-pioglitazone    int32
change                    int32
diabetesMed               int32
target  

In [15]:
data = data.reset_index(drop=True)

In [16]:
data[data['target'] == 0]

,race,gender,age,admission_type_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,coverByInsurance,f_diag,s_diag,t_diag,clustering3,clustering4,clustering2,medical_specialty_new,discharge_id,admission_id
0,2,1,5,1,10,45,1,23,0,0,...,1,4,16,16,0,1,1,1,8,6
1,2,0,5,1,2,39,3,12,0,0,...,0,17,11,11,2,0,0,0,0,0
2,0,1,6,0,6,45,1,22,0,0,...,0,11,4,10,2,0,0,4,0,6
3,2,0,6,5,2,72,2,19,0,0,...,0,17,5,7,2,0,0,5,0,1
4,2,0,7,5,10,56,2,24,0,0,...,0,4,4,11,2,0,0,8,4,0
5,4,0,6,2,3,35,2,10,1,0,...,0,12,16,11,0,2,0,8,0,0
6,2,1,8,0,5,51,0,17,0,0,...,0,9,4,4,0,1,1,5,5,6
7,2,0,6,2,4,63,0,27,0,1,...,0,9,9,11,1,3,1,4,0,0
8,2,1,7,2,1,38,1,3,0,0,...,0,5,3,11,0,2,0,3,0,0
9,2,1,6,0,2,33,5,16,0,2,...,0,4,4,11,0,2,0,5,0,6


In [17]:
unadmitted = data[:74816]

In [18]:
unadmitted

,race,gender,age,admission_type_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,coverByInsurance,f_diag,s_diag,t_diag,clustering3,clustering4,clustering2,medical_specialty_new,discharge_id,admission_id
0,2,1,5,1,10,45,1,23,0,0,...,1,4,16,16,0,1,1,1,8,6
1,2,0,5,1,2,39,3,12,0,0,...,0,17,11,11,2,0,0,0,0,0
2,0,1,6,0,6,45,1,22,0,0,...,0,11,4,10,2,0,0,4,0,6
3,2,0,6,5,2,72,2,19,0,0,...,0,17,5,7,2,0,0,5,0,1
4,2,0,7,5,10,56,2,24,0,0,...,0,4,4,11,2,0,0,8,4,0
5,4,0,6,2,3,35,2,10,1,0,...,0,12,16,11,0,2,0,8,0,0
6,2,1,8,0,5,51,0,17,0,0,...,0,9,4,4,0,1,1,5,5,6
7,2,0,6,2,4,63,0,27,0,1,...,0,9,9,11,1,3,1,4,0,0
8,2,1,7,2,1,38,1,3,0,0,...,0,5,3,11,0,2,0,3,0,0
9,2,1,6,0,2,33,5,16,0,2,...,0,4,4,11,0,2,0,5,0,6


In [19]:
admitted = data[74817:]

In [22]:
admitted

,race,gender,age,admission_type_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,coverByInsurance,f_diag,s_diag,t_diag,clustering3,clustering4,clustering2,medical_specialty_new,discharge_id,admission_id
74817,2,1,7,0,11,57,3,22,1,1,...,0,14,10,4,0,2,0,5,8,6
74818,2,0,6,0,12,74,1,23,0,0,...,0,4,5,4,2,0,0,1,0,6
74819,2,1,7,0,1,44,0,17,2,3,...,0,4,4,11,1,3,1,5,2,6
74820,2,1,4,2,1,45,1,9,0,0,...,0,4,10,10,2,0,0,9,0,0
74821,0,1,7,0,3,43,0,19,0,1,...,0,9,11,4,2,0,0,3,5,4
74822,2,0,9,1,3,46,0,7,0,0,...,0,9,9,11,1,3,1,1,5,6
74823,2,1,5,2,1,10,6,12,0,0,...,1,4,4,17,0,1,1,5,0,0
74824,2,1,3,1,4,54,0,27,0,13,...,0,9,15,3,1,1,1,1,0,6
74825,0,0,6,0,8,37,0,24,0,0,...,0,6,6,13,2,2,0,3,8,6
74826,0,0,6,0,8,57,0,15,0,0,...,0,6,11,4,0,2,0,5,8,6


In [23]:
holdout1 = unadmitted.sample(frac = 0.15)

In [24]:
data1 = unadmitted.drop(holdout1.index)

In [25]:
data1.shape

(63594, 44)

In [26]:
unadmitted.shape

(74816, 44)

In [27]:
holdout1.shape

(11222, 44)

In [28]:
holdout2 = admitted.sample(frac = 0.15)

In [29]:
admitted.shape

(9615, 44)

In [30]:
data2 = admitted.drop(holdout2.index)

In [31]:
data2.shape

(8173, 44)

In [32]:
holdout2.shape

(1442, 44)

In [33]:
holdout = holdout1.append(holdout2, ignore_index = True)

In [34]:
holdout = holdout.sample(frac=1).reset_index(drop=True)

In [35]:
new_data = data1.append(data2, ignore_index = True)

In [36]:
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [122]:
#holdout = holdout.drop('Unnamed: 0', axis = 1)

In [123]:
#new_data = new_data.drop('Unnamed: 0', axis = 1)

In [38]:
holdout.to_csv('./final_data/holdout.csv')

In [42]:
new_data.to_csv('./final_data/new_data.csv')

In [40]:
holdout.shape

(12664, 44)

In [41]:
new_data.shape

(71767, 44)

#====================clean up holdout

In [61]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import random
import re

In [62]:
holdout = pd.read_csv('./dataForTeammates/holdout.csv')
print(holdout.shape)
holdout.columns

(15265, 55)


Index(['Unnamed: 0', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'f_diag', 's_diag', 't_diag',
       'clustering3', 'clustering4', 'clustering2'],


In [63]:
holdout.dtypes

Unnamed: 0                   int64
encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide         

In [64]:
holdout = holdout.drop(['Unnamed: 0','encounter_id', 'patient_nbr'], axis = 1)

In [65]:
dead_ids = [11, 13, 14, 19,20,21]
holdout = holdout[~holdout['discharge_disposition_id'].isin(dead_ids)]

In [66]:
holdout['admission_type_id'] = holdout['admission_type_id'].astype(str)
holdout['discharge_disposition_id'] = holdout['discharge_disposition_id'].astype(str)
holdout['admission_source_id'] = holdout['admission_source_id'].astype(str)

In [67]:
top10_medical = ['No', 'InternalMedicine', 'Emergency/Trauma','Family/GeneralPractice','Cardiology','Surgery-General','Nephrology','Orthopedics','Orthopedics-Reconstructive']

In [68]:
holdout.dtypes

race                        object
gender                      object
age                         object
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
time_in_hospital             int64
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone               object
acarbose            

In [69]:
holdout['medical_specialty_new'] = holdout['medical_specialty'].copy()


In [70]:
holdout.loc[~holdout['medical_specialty_new'].isin(top10_medical), 'medical_specialty_new'] = 'Other'
holdout = holdout.drop('medical_specialty', axis = 1)

In [71]:
holdout.medical_specialty_new.value_counts()

No                            7276
InternalMedicine              2171
Other                         1444
Emergency/Trauma              1120
Family/GeneralPractice        1072
Cardiology                     784
Surgery-General                460
Nephrology                     220
Orthopedics-Reconstructive     182
Orthopedics                    181
Name: medical_specialty_new, dtype: int64

In [72]:
holdout['discharge_id'] = holdout['discharge_disposition_id'].copy()
top10_discharge = ['1', '3', '6','18','2','22','5','25','4']
holdout.loc[~holdout['discharge_id'].isin(top10_discharge), 'discharge_id'] = 'Other'
holdout = holdout.drop('discharge_disposition_id', axis = 1)

In [73]:
holdout.discharge_id.value_counts()

1        8954
3        2081
6        2021
18        590
2         313
22        286
Other     233
5         180
25        146
4         106
Name: discharge_id, dtype: int64

In [74]:
holdout['admission_id'] = holdout['admission_source_id'].copy()
top10_admission = ['7', '1', '17','4','6','2','5']
holdout.loc[~holdout['admission_id'].isin(top10_admission), 'admission_id'] = 'Other'
holdout = holdout.drop('admission_source_id', axis = 1)

In [75]:
holdout.dtypes

race                        object
gender                      object
age                         object
admission_type_id           object
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone               object
acarbose                    object
miglitol                    object
troglitazone                object
tolazamide          

In [76]:
holdout.shape

(14910, 52)

In [77]:
holdout.to_csv('last_holdout.csv', index = False)